In [1]:
#EDA for recipe databases

In [2]:
#--> Dataset: Recipe NLG: https://www.kaggle.com/datasets/paultimothymooney/recipenlg

In [3]:
#EDA Tasks
#0. Overall understanding of the structure of the dataset
#1. Count of recipes
#2. Avg # of ingredients per recipe 
#3. # of unique ingredients
#4. List of unique ingredients

In [4]:
common_food_items = ['salt', 'kosher salt', 'onion powder', 'garlic', 'garlic powder', 'garlic salt', 
                     'freshly ground black pepper', 'fresh ground black pepper',
                     'butter', 'salted butter', 'unsalted butter', 
                     'oil', 'olive oil', 'vegetable oil', 'extra-virgin olive oil', 'extra virgin olive oil',
                     'cooking oil', 'salad oil', 
                     'flour', 'whole wheat flour', 'light brown sugar',
                     'sugar', 'brown sugar', 'white sugar', 'honey', 'maple syrup', 
                     'balsamic vinegar','red wine vinegar', 'white wine vinegar', 'white vinegar', 'cider vinegar',
                     'soy sauce', 'worcestershire sauce',
                     'dijon mustard', 'ketchup', 'mayonnaise', 'hot sauce', 'mustard', 'barbecue sauce', 
                     'pepper', 'paprika', 'cumin', 'cilantro', 'thyme', 'cayenne pepper', 'chili powder', 'oregano', 
                     'rosemary', 'coriander', 'cinnamon', 'ground cinnamon', 'nutmeg', 'ginger', 'allspice', 
                     'ground cumin', 'bay leaf', 'ground ginger', 'ground nutmeg', 'italian seasoning',
                     'bay leaves', 'dill', 'tumeric', 'beef broth', 'basil', 'parsley', 'marjoram', 'sage', 'mustard powder',
                     'vegetable broth', 'vanilla extract', 'vanilla', 'baking powder', 'baking soda', 
                     'black pepper', 'cornstarch', 'water', 'boiling water', 'cold water',
                     '\\u00bc', 'all purpose', 'all-purpose']

In [5]:
import pandas as pd

df = pd.read_csv('RecipeNLG_dataset.csv')

In [6]:
df.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [7]:
listoflist = df['NER'].tolist()

In [8]:
array = df['NER'].to_numpy()

In [9]:
array

array(['["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"]',
       '["beef", "chicken breasts", "cream of mushroom soup", "sour cream"]',
       '["frozen corn", "cream cheese", "butter", "garlic powder", "salt", "pepper"]',
       ...,
       '["radish", "Sesame oil", "White sesame seeds", "Salt", "Soy sauce"]',
       '["apple cider", "sugar", "kosher salt", "bay leaves", "arbol", "berries", "caraway seeds", "mustard seeds", "coriander seeds", "pork chops", "Freshly ground black pepper", "neutral oil", "unsalted butter", "shallots", "thyme", "brandy", "heavy cream", "pork", "grain mustard", "horseradish", "apple cider", "cinnamon", "apples", "egg", "water", "rice flour", "all-purpose", "neutral oil"]',
       '["ground veal", "sausage", "bread crumbs", "milk", "garlic", "flat-leaf", "egg whites", "tomato paste", "freshly grated Pecorino Romano cheese", "Salt", "tomato sauce", "red pepper"]'],
      dtype=object)

In [10]:
array[0]

'["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"]'

In [11]:
#Begin with an empty list
#Within each array, remove the quotations, remove the left and right brackets, and split into a list 
#Add the cleaned array to the new list 

In [12]:
len(array)

2231142

In [13]:
cleaned_list = []

for i in range(2231142):
    phrase = array[i]
    test2 = phrase.replace('"','')
    test3 = test2.replace('[','')
    test4 = test3.replace(']','')
    phrase_to_list_cleaned = test4.split(',')
    cleaned_list.append(phrase_to_list_cleaned)

In [14]:
cleaned_list

[['brown sugar',
  ' milk',
  ' vanilla',
  ' nuts',
  ' butter',
  ' bite size shredded rice biscuits'],
 ['beef', ' chicken breasts', ' cream of mushroom soup', ' sour cream'],
 ['frozen corn',
  ' cream cheese',
  ' butter',
  ' garlic powder',
  ' salt',
  ' pepper'],
 ['chicken', ' chicken gravy', ' cream of mushroom soup', ' shredded cheese'],
 ['peanut butter',
  ' graham cracker crumbs',
  ' butter',
  ' powdered sugar',
  ' chocolate chips'],
 ['baking potatoes',
  ' extra lean ground beef',
  ' butter',
  ' milk',
  ' salt',
  ' pepper',
  ' Cheddar cheese',
  ' bacon',
  ' green onion',
  ' sour cream'],
 ['sugar',
  ' butter',
  ' egg',
  ' buttermilk',
  ' flour',
  ' salt',
  ' soda',
  ' buttermilk',
  ' rhubarb',
  ' vanilla'],
 ['cream-style corn',
  ' whole kernel corn',
  ' crackers',
  ' egg',
  ' butter',
  ' pepper'],
 ['tomatoes',
  ' water',
  ' onions',
  ' Worcestershire sauce',
  ' green peppers',
  ' oil'],
 ['pineapple',
  ' condensed milk',
  ' lemons',
  

In [15]:
len(cleaned_list)

2231142

In [16]:
#Remove the spaces in front of each word
cleaned_list_2 = [[item.strip().lower() for item in sub_list] for sub_list in cleaned_list]

In [17]:
len(cleaned_list_2)

2231142

In [18]:
#Remove common household items from each ingredient list 
cleaned_list_3 = [[item for item in sublist if item not in common_food_items] for sublist in cleaned_list_2]

In [19]:
#Count length of each recipe; store count and continue to add to it
#Divide by 2231142

sum_of_list = 0

for i in range(2231142):
    item = len(cleaned_list[i])
    sum_of_list += item


In [20]:
average_num_ingredients = sum_of_list / len(cleaned_list)
average_num_ingredients

8.480955940948625

In [21]:
def flatten_comprehension(matrix):
    return [item for row in matrix for item in row]

In [22]:
complete_ingredient_list = flatten_comprehension(cleaned_list_3)

In [23]:
complete_ingredient_list

['milk',
 'nuts',
 'bite size shredded rice biscuits',
 'beef',
 'chicken breasts',
 'cream of mushroom soup',
 'sour cream',
 'frozen corn',
 'cream cheese',
 'chicken',
 'chicken gravy',
 'cream of mushroom soup',
 'shredded cheese',
 'peanut butter',
 'graham cracker crumbs',
 'powdered sugar',
 'chocolate chips',
 'baking potatoes',
 'extra lean ground beef',
 'milk',
 'cheddar cheese',
 'bacon',
 'green onion',
 'sour cream',
 'egg',
 'buttermilk',
 'soda',
 'buttermilk',
 'rhubarb',
 'cream-style corn',
 'whole kernel corn',
 'crackers',
 'egg',
 'tomatoes',
 'onions',
 'green peppers',
 'pineapple',
 'condensed milk',
 'lemons',
 'pecans',
 'graham cracker crusts',
 'dark sweet pitted cherries',
 'ginger ale',
 'flavor gelatin',
 'almond extract',
 'marshmallows',
 'powdered sugar',
 'peanut butter',
 'paraffin',
 'chocolate chips',
 'chicken',
 'taco sauce',
 'sour cream',
 'cream cheese',
 'ground beef',
 'taco seasoning',
 'shredded lettuce',
 'tomato',
 'onion',
 'green pepp

In [24]:
#Create a unique list of ingredients

set_ingredients = set(complete_ingredient_list) 
list_unique_ingredients = (list(set_ingredients))

In [25]:
len(list_unique_ingredients)

198519

In [26]:
# Specify the file path
file_path = 'ingredient_list_v2.txt'

#Open the file in write mode ('w')
with open(file_path, 'w') as file:
    for item in list_unique_ingredients:
        file.write(str(item) + '\n')


# Confirmation message
print(f'The list has been written to {file_path}')

The list has been written to ingredient_list_v2.txt


In [27]:
from collections import Counter

word_counts = Counter(complete_ingredient_list)

In [28]:
sorted_ingredient_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

In [29]:
sorted_ingredient_counts

[('eggs', 422212),
 ('onion', 391271),
 ('milk', 370287),
 ('egg', 176198),
 ('tomatoes', 174311),
 ('lemon juice', 155456),
 ('sour cream', 122732),
 ('cream cheese', 121520),
 ('celery', 116653),
 ('margarine', 114909),
 ('chicken', 114507),
 ('cheddar cheese', 111260),
 ('onions', 102596),
 ('parmesan cheese', 92833),
 ('pecans', 83320),
 ('potatoes', 81824),
 ('carrots', 80450),
 ('nuts', 79693),
 ('pineapple', 76432),
 ('bacon', 72351),
 ('chicken broth', 68993),
 ('ground beef', 67258),
 ('mushrooms', 64500),
 ('powdered sugar', 60630),
 ('green pepper', 59923),
 ('vinegar', 59833),
 ('red pepper', 59451),
 ('green onions', 57695),
 ('lemon', 56133),
 ('shortening', 53287),
 ('walnuts', 52666),
 ('chicken breasts', 52312),
 ('rice', 52231),
 ('heavy cream', 48668),
 ('buttermilk', 48426),
 ('raisins', 47241),
 ('clove garlic', 46163),
 ('cocoa', 45707),
 ('cheese', 45106),
 ('red onion', 44990),
 ('orange juice', 44234),
 ('soda', 43373),
 ('tomato sauce', 43048),
 ('bread', 4290